In [100]:
# ------------------------------------------------ ### PROBLEMA DA DIETA ### ------------------------------------------------

#       Tenta resolver o problema de dado N elementos e M caracteristicas escolher K porções de cada elemento tal que a seguinte 
# lei não é violada:
# => Somatório[i até N] (xi*mij) <= ou >= Tj (xi é o K do elemento i, e mij é a j-éssima caracteristica, indo até M, do i-éssimo 
# elemento) e Tj é um valor dado. Isso deve ser verdade para todas as M características dadas.

import numpy as np
import math
from random import randrange, uniform
from scipy.optimize import linprog

In [101]:
# Vamos criar um exemplo de problema, será importante para nós no futuro!

numberElements = 5
numberCharacteristcs = 10
space = 100

file = open('problem ' + str(numberElements) + '-' + str(numberCharacteristcs) + '.txt', 'w')

file.write(str(numberElements) + '\n')
file.write(str(numberCharacteristcs) + '\n')
for i in range(numberElements):
    for j in range(numberCharacteristcs):
        file.write(str(randrange(0, space)) + ' ')
    file.write(str(randrange(0, space / 10)))
    file.write('\n')
for j in range(numberCharacteristcs + 1):
        file.write(str(randrange(space * 4, space * 10)) + ' ')

file.close()

In [102]:
# Vamos ler agora os exemplos criados

#file = open('problem ' + str(numberElements) + '-' + str(numberCharacteristcs) + '.txt', 'r')
file = open('real problem.txt' , 'r')

nE = int(file.readline())
nC = int(file.readline())

matrix = np.zeros((nE+1,nC+1))
for i in range(nE + 1):
    aux = (file.readline().split())
    matrix[i] = [float(item) for item in aux]
matrix[-1][-1] = 0

print(matrix) 

[[4.6190e+01 8.0000e-02 9.5100e+00 1.5700e+00 2.5000e+00]
 [1.7080e+01 4.0000e-02 2.4900e+00 1.6900e+00 3.0000e+00]
 [8.0000e-02 1.0000e-02 2.4000e-01 6.0000e-02 5.0000e-01]
 [4.6110e+01 6.0000e-01 6.2000e-01 8.0000e-01 2.5000e+00]
 [4.8200e+01 2.0200e+00 2.4770e+01 7.7000e-01 3.0000e+00]
 [1.6785e+02 1.3630e+01 3.1000e-01 1.1020e+01 4.0000e+00]
 [2.6800e+02 5.4100e+00 4.1740e+01 1.3650e+01 0.0000e+00]]


In [103]:
# Após ler os exemplos vamos achar uma resposta para a solução, primeiro de tudo vamos setar as listas a seguir

variables = [0] * nE # Lista das variáveis a serem usadas
constraints = [0] * nC # Lista de restrições, ou seja, xi*mij => variables[i]*matrix[i][j]

# Função para decidir se é viável
def isViable(constraint, j):
    return constraint <= matrix[nE][j]

# Vamos criar uma função para calcular a lista dos constraints
def calcConstraints():
    for j in range(nC):
        aux = 0
        for i in range(nE):
            flag = True
            while flag:
                aux = aux + (variables[i] * matrix[i][j])
                if isViable(aux, j):
                    flag = False
                else:
                    variables[i] = uniform(0, space)
        constraints[j] = aux
    return constraints

# E agora a função objetiva
def calcObjFunction():
    aux = 0
    for i in range(nE):
        aux = aux + (matrix[i][nC] * variables[i])
    return aux

def inv(x):
    return x * (-1)

In [106]:
#Agora vamos tentar achar a solução por tentativa, queremos a menor função objetiva!!

# minimizar c quando A_lb >= b, adaptando
# minimizar c quando A_ub * (-1) <= b * (-1)

# Tratando c
# Valores de c = [x[nC] for x in matrix]
# A função optmizations.linprog() sempre minimiza

objFun = [x[nC] for x in matrix]
del objFun[-1]
#objFun = [i*(-1) for i in objFun] #caso queiramos maximizar

print(f'Funcao Objetiva: {objFun}')

#Tratando A_lb
#Valores de A_lb = matrix[0:nE][0:nC-1])
A_lb = list()

for j in range(nC):
    A_lb.append(list())
    for i in range(nE):
        A_lb[j].append(inv(matrix[i][j])) #função inv inverte o valor da A, pois a inequação é >=.

print(f'Coeficientes: {A_lb}')

#Tratando b
#Valor de b = matrix[nE][0:nC]

b = list()
for i in matrix[nE]:
    b.append(inv(i)) #função inv inverte o valor da b, pois a inequação é >=.
    
del b[-1]
print(f'Valores Nutricionais Mínimos: {b}')

#Tratando os limites
x_bnds = list()

for i in range(nE):
    x_bnds.append((0.1, None))
x_bnds[2] = (0.5, None)
x_bnds[5] = (0, 1)
print(f'Limites {x_bnds}')

res = linprog(objFun, A_lb, b, bounds=(x_bnds))
print(res)        

Funcao Objetiva: [2.5, 3.0, 0.5, 2.5, 3.0, 4.0]
Coeficientes: [[-46.19, -17.08, -0.08, -46.11, -48.2, -167.85], [-0.08, -0.04, -0.01, -0.6, -2.02, -13.63], [-9.51, -2.49, -0.24, -0.62, -24.77, -0.31], [-1.57, -1.69, -0.06, -0.8, -0.77, -11.02]]
Valores Nutricionais Mínimos: [-268.0, -5.41, -41.74, -13.65]
Limites [(0.1, None), (0.1, None), (0.5, None), (0.1, None), (0.1, None), (0, 1)]
     fun: 10.904436390823102
 message: 'Optimization terminated successfully.'
     nit: 12
   slack: array([ 9.37505868, 11.04493044,  0.        ,  0.        ,  0.74473392,
        0.        ,  0.        ,  0.        ,  1.23086719,  0.        ])
  status: 0
 success: True
       x: array([0.84473392, 0.1       , 0.5       , 0.1       , 1.33086719,
       1.        ])


In [109]:
file = open('result.txt', 'w')

file.write(str(res))
print("Archive printed in .txt successfully!!")

file.close()

Archive printed in .txt successfully!!
